# Lowest Permeability Membrane Compositions
### Which experimentally tested membranes block phenol and m-cresol best — and where do they sit in the 4-component composition space?

This notebook ranks the measured membranes by permeability and shows that the top performers
are **not all in the same region** of the composition space. Different chemistry works for different molecules.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec

# ── Colour palette ────────────────────────────────────────────────────────
COMP_COLORS = {
    'Sparsa 1 (27G26)':   '#3498db',
    'Sparsa 2 (30G25)':   '#2ecc71',
    'Carbosil 1 (2080A)': '#e74c3c',
    'Carbosil 2 (2090A)': '#f39c12',
}
COMP_KEYS = list(COMP_COLORS.keys())

# ── Data ──────────────────────────────────────────────────────────────────
PHENOL_DATA = [
    {'id':'M-01',     'S1':100,'S2':  0,'C1':  0,'C2':  0,'P':1.60618e-6},
    {'id':'M-02',     'S1':  0,'S2':100,'C1':  0,'C2':  0,'P':7.55954e-7},
    {'id':'M-03',     'S1':  0,'S2':  0,'C1':100,'C2':  0,'P':1.68063e-7},
    {'id':'M-03(3)',  'S1':  0,'S2':  0,'C1':100,'C2':  0,'P':2.17540e-7},
    {'id':'M-04(2)',  'S1':  0,'S2':  0,'C1':  0,'C2':100,'P':1.4470e-7},
    {'id':'M-04(3)',  'S1':  0,'S2':  0,'C1':  0,'C2':100,'P':1.4094e-7},
    {'id':'M-05',     'S1': 60,'S2': 40,'C1':  0,'C2':  0,'P':5.75051e-7},
    {'id':'M-07',     'S1': 30,'S2': 70,'C1':  0,'C2':  0,'P':3.39749e-8},
    {'id':'M-11',     'S1': 10,'S2': 20,'C1': 70,'C2':  0,'P':1.59367e-7},
    {'id':'M-13',     'S1': 40,'S2': 30,'C1': 10,'C2': 20,'P':1.2747e-6},
    {'id':'M-20',     'S1':  0,'S2': 60,'C1': 20,'C2': 20,'P':5.4426e-7},
]

MCRESOL_DATA = [
    {'id':'M-02',      'S1':  0,'S2':100,'C1':  0,'C2':  0,'P':1.02150e-7},
    {'id':'M-03(2)',   'S1':  0,'S2':  0,'C1':100,'C2':  0,'P':7.6622e-8},
    {'id':'M-04(2)',   'S1':  0,'S2':  0,'C1':  0,'C2':100,'P':1.46250e-7},
    {'id':'M-07',      'S1': 30,'S2': 70,'C1':  0,'C2':  0,'P':9.75280e-8},
    {'id':'M-09',      'S1': 60,'S2': 10,'C1': 30,'C2':  0,'P':5.05120e-8},
    {'id':'M-11',      'S1': 10,'S2': 20,'C1': 70,'C2':  0,'P':1.09746e-7},
    {'id':'M-13(2a)',  'S1': 40,'S2': 30,'C1': 10,'C2': 20,'P':2.1499e-7},
    {'id':'M-13(2b)',  'S1': 40,'S2': 30,'C1': 10,'C2': 20,'P':1.5607e-7},
    {'id':'M-14',      'S1': 20,'S2': 40,'C1':  0,'C2': 40,'P':6.9491e-8},
    {'id':'M-15',      'S1':  0,'S2': 50,'C1': 50,'C2':  0,'P':1.81053e-7},
    {'id':'M-20',      'S1':  0,'S2': 60,'C1': 20,'C2': 20,'P':1.82590e-7},
    {'id':'M-22',      'S1': 37,'S2': 63,'C1':  0,'C2':  0,'P':4.13480e-7},
    {'id':'M-23',      'S1': 25,'S2': 35,'C1': 40,'C2':  0,'P':1.4204e-7},
]

ph = pd.DataFrame(PHENOL_DATA).sort_values('P').reset_index(drop=True)
mc = pd.DataFrame(MCRESOL_DATA).sort_values('P').reset_index(drop=True)

print('Phenol — ranked by permeability:')
print(ph[['id','S1','S2','C1','C2','P']].to_string(index=False))
print()
print('M-Cresol — ranked by permeability:')
print(mc[['id','S1','S2','C1','C2','P']].to_string(index=False))

---
## Top 3 Lowest Permeability Compositions

Rather than showing all membranes, we pull the **3 unique lowest-permeability compositions** for each molecule.
Where two membranes share the same composition (replicates), they are merged and averaged.

In [ ]:
def top3_unique(df, n=3):
    """Return top-n unique compositions (merge replicates by averaging P)."""
    # Group by composition, take mean P
    grouped = df.groupby(['S1','S2','C1','C2']).agg(
        P=('P','mean'),
        ids=('id', lambda x: ' / '.join(x))
    ).reset_index().sort_values('P')
    return grouped.head(n).reset_index(drop=True)

top_ph = top3_unique(ph)
top_mc = top3_unique(mc)

print('=== Top 3 Phenol ===')
for i, row in top_ph.iterrows():
    print(f'  #{i+1}  {row["ids"]:20s}  S1={int(row.S1):3d}% S2={int(row.S2):3d}% C1={int(row.C1):3d}% C2={int(row.C2):3d}%  P = {row.P:.3e} cm/s')

print()
print('=== Top 3 M-Cresol ===')
for i, row in top_mc.iterrows():
    print(f'  #{i+1}  {row["ids"]:20s}  S1={int(row.S1):3d}% S2={int(row.S2):3d}% C1={int(row.C1):3d}% C2={int(row.C2):3d}%  P = {row.P:.3e} cm/s')

---
## Composition Breakdown — What Each Top Membrane Is Made Of

Each stacked bar shows the 4-component recipe for a membrane. If the top compositions were all the same,
the bars would look identical. They don't.

In [ ]:
def composition_bars(ax, top_df, mol_name, mol_color):
    """Stacked bar chart showing composition of each top membrane."""
    cols_data = ['S1','S2','C1','C2']
    bar_labels = [f'#{i+1}\n{row["ids"]}\n{row["P"]:.2e} cm/s'
                  for i, row in top_df.iterrows()]

    x = np.arange(len(top_df))
    bottom = np.zeros(len(top_df))

    for col, label, color in zip(cols_data, COMP_KEYS, COMP_COLORS.values()):
        vals = top_df[col].values
        bars = ax.bar(x, vals, bottom=bottom, color=color, label=label,
                      edgecolor='#1a1a1a', linewidth=0.8, width=0.55)
        # Label each segment if large enough
        for bar, v in zip(bars, vals):
            if v >= 8:
                yc = bar.get_y() + bar.get_height() / 2
                ax.text(bar.get_x() + bar.get_width()/2, yc,
                        f'{int(v)}%', ha='center', va='center',
                        fontsize=10, color='white', fontweight='bold')
        bottom += vals

    ax.set_xticks(x)
    ax.set_xticklabels(bar_labels, fontsize=9)
    ax.set_ylim(0, 115)
    ax.set_ylabel('Component fraction (%)')
    ax.set_title(f'{mol_name} — Top 3 Lowest Permeability', color='#e0e0ff', fontsize=12)
    ax.axhline(100, color='#ffffff30', lw=1, ls='--')
    ax.grid(axis='y', alpha=0.2)


fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.patch.set_facecolor('#1a1a2e')
for ax in axes:
    ax.set_facecolor('#16213e')
    for spine in ax.spines.values():
        spine.set_edgecolor('#444')
    ax.tick_params(colors='#ccc')
    ax.yaxis.label.set_color('#ccc')

composition_bars(axes[0], top_ph, 'Phenol',   '#e74c3c')
composition_bars(axes[1], top_mc, 'M-Cresol', '#9b59b6')

# Shared legend
handles = [mpatches.Patch(color=c, label=l) for l, c in COMP_COLORS.items()]
fig.legend(handles=handles, loc='lower center', ncol=4, fontsize=9,
           facecolor='#1a1a2e', edgecolor='#555', labelcolor='#ddd',
           bbox_to_anchor=(0.5, -0.05))

plt.suptitle('Top 3 lowest-permeability membranes — composition breakdown',
             color='#e0e0ff', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

---
## Where Do They Sit in the 4D Composition Space?

The 4 components lie on a 3-simplex (they must sum to 100%). We can't draw 4D space directly,
but two complementary 2D projections give an honest picture of where each membrane sits:
- **Left**: Sparsa 1 vs Sparsa 2 fraction (within the Sparsa sub-blend)
- **Right**: Carbosil 1 vs Carbosil 2 fraction (within the Carbosil sub-blend)

All membranes are shown as grey dots; top-3 for each molecule are highlighted.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.patch.set_facecolor('#1a1a2e')

all_data = []
for row in PHENOL_DATA:
    all_data.append({**row, 'mol':'Phenol'})
for row in MCRESOL_DATA:
    # deduplicate by id (keep first occurrence)
    if row['id'] not in [d['id'] for d in all_data]:
        all_data.append({**row, 'mol':'M-Cresol'})
all_df = pd.DataFrame(all_data)

configs = [
    (axes[0], 'S1', 'S2', 'Sparsa 1 (%)', 'Sparsa 2 (%)'),
    (axes[1], 'C1', 'C2', 'Carbosil 1 (%)', 'Carbosil 2 (%)'),
]

# Markers for rank
rank_markers = {1: '*', 2: 'D', 3: 's'}
rank_sizes   = {1: 280, 2: 140, 3: 120}

mol_colors = {'Phenol': '#e74c3c', 'M-Cresol': '#9b59b6'}

for ax, xcol, ycol, xlabel, ylabel in configs:
    ax.set_facecolor('#16213e')
    for spine in ax.spines.values():
        spine.set_edgecolor('#444')
    ax.tick_params(colors='#ccc')

    # All membranes in grey
    ax.scatter(all_df[xcol], all_df[ycol], color='#555', s=50,
               edgecolors='#888', lw=0.5, zorder=2, label='All membranes')
    for _, row in all_df.iterrows():
        ax.annotate(row['id'], (row[xcol], row[ycol]),
                    textcoords='offset points', xytext=(4, 3),
                    fontsize=6, color='#777', zorder=3)

    # Top-3 for each molecule
    for mol, top_df, col in [('Phenol', top_ph, '#e74c3c'), ('M-Cresol', top_mc, '#9b59b6')]:
        for rank, (_, row) in enumerate(top_df.iterrows(), start=1):
            ax.scatter(row[xcol], row[ycol],
                       color=col, s=rank_sizes[rank],
                       marker=rank_markers[rank],
                       edgecolors='white', lw=1.2,
                       zorder=6)
            ax.annotate(f'{mol[:2]}#{rank}',
                        (row[xcol], row[ycol]),
                        textcoords='offset points', xytext=(6, 5),
                        fontsize=8.5, color='white',
                        fontweight='bold', zorder=7)

    ax.set_xlabel(xlabel, color='#ccc')
    ax.set_ylabel(ylabel, color='#ccc')
    ax.set_xlim(-5, 105)
    ax.set_ylim(-5, 105)
    ax.grid(True, alpha=0.15)

axes[0].set_title('Sparsa sub-space projection', color='#e0e0ff', fontsize=11)
axes[1].set_title('Carbosil sub-space projection', color='#e0e0ff', fontsize=11)

# Legend
from matplotlib.lines import Line2D
legend_handles = [
    Line2D([0],[0], marker='o',  color='#555', ms=7,  lw=0, label='All membranes (grey)'),
    Line2D([0],[0], marker='*',  color='w',    ms=12, lw=0, label='#1 (best)'),
    Line2D([0],[0], marker='D',  color='w',    ms=8,  lw=0, label='#2'),
    Line2D([0],[0], marker='s',  color='w',    ms=8,  lw=0, label='#3'),
    mpatches.Patch(color='#e74c3c', label='Phenol top-3'),
    mpatches.Patch(color='#9b59b6', label='M-Cresol top-3'),
]
fig.legend(handles=legend_handles, loc='lower center', ncol=6, fontsize=9,
           facecolor='#1a1a2e', edgecolor='#555', labelcolor='#ddd',
           bbox_to_anchor=(0.5, -0.06))

plt.suptitle('Top-3 compositions in 2D projections of the 4-component space\n'
             '(Ph=Phenol, MC=M-Cresol; #1 = ★  #2 = ◆  #3 = ■)',
             color='#e0e0ff', fontsize=12, y=1.03)
plt.tight_layout()
plt.show()

---
## Parallel Coordinates — Full 4D View

Parallel coordinates plot each composition as a line connecting its values on 4 axes (S1, S2, C1, C2).
Lines that are very different in shape = compositions that are far apart in the 4D space.
All membranes are shown in grey; top-3 for each molecule are coloured.

In [ ]:
fig, ax = plt.subplots(figsize=(13, 6))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#16213e')
for spine in ax.spines.values():
    spine.set_edgecolor('#444')
ax.tick_params(colors='#ccc')

comp_axes = ['S1', 'S2', 'C1', 'C2']
comp_labels = ['Sparsa 1\n(27G26)', 'Sparsa 2\n(30G25)', 'Carbosil 1\n(2080A)', 'Carbosil 2\n(2090A)']
x_pos = np.arange(4)

# Collect all unique compositions from both datasets
seen = set()
all_uniq = []
for row in PHENOL_DATA + MCRESOL_DATA:
    key = (row['S1'], row['S2'], row['C1'], row['C2'])
    if key not in seen:
        seen.add(key)
        all_uniq.append(row)

# Draw all in grey first
for row in all_uniq:
    vals = [row[c] for c in comp_axes]
    ax.plot(x_pos, vals, color='#444', lw=1, alpha=0.6, zorder=1)

# Draw top-3 for each molecule
rank_lw    = {1: 3.5, 2: 2.5, 3: 2.0}
rank_alpha = {1: 1.0, 2: 0.85, 3: 0.70}
rank_ls    = {1: '-', 2: '--', 3: ':'}

for mol, top_df, col, offset in [('Phenol', top_ph, '#e74c3c', -0.04),
                                   ('M-Cresol', top_mc, '#9b59b6', 0.04)]:
    for rank, (_, row) in enumerate(top_df.iterrows(), start=1):
        vals = [row[c] for c in comp_axes]
        ax.plot(x_pos + offset, vals,
                color=col, lw=rank_lw[rank], alpha=rank_alpha[rank],
                ls=rank_ls[rank], zorder=5+rank,
                label=f'{mol} #{rank} ({row["ids"]})')
        # Annotate the last axis
        ax.annotate(f'{mol[:2]}#{rank}  {row.P:.1e}',
                    (3.02, vals[3]),
                    fontsize=7.5, color=col, va='center')

# Vertical axis lines
for xi, label in zip(x_pos, comp_labels):
    ax.axvline(xi, color='#888', lw=1.2, zorder=2)
    ax.text(xi, 105, label, ha='center', va='bottom',
            fontsize=9, color='#ddd')

ax.set_xlim(-0.2, 3.35)
ax.set_ylim(-5, 115)
ax.set_xticks(x_pos)
ax.set_xticklabels([''] * 4)
ax.set_ylabel('Component fraction (%)', color='#ccc')
ax.set_title('Parallel coordinates — top-3 lowest permeability compositions for Phenol and M-Cresol\n'
             'Grey lines = all other tested membranes',
             color='#e0e0ff', fontsize=11)
ax.legend(loc='upper left', fontsize=7.5, facecolor='#1a1a2e',
          edgecolor='#555', labelcolor='#ddd', ncol=2)
ax.grid(axis='y', alpha=0.1)
plt.tight_layout()
plt.show()

---
## Joint Ranking — Which Membranes Are Good for Both Molecules?

A membrane that blocks both phenol AND m-cresol well is the real target.
This plot ranks every membrane by its **normalised combined score** (lower = better for both).

In [ ]:
# Find membranes measured for BOTH molecules (same S1,S2,C1,C2)
ph_df = pd.DataFrame(PHENOL_DATA)
mc_df = pd.DataFrame(MCRESOL_DATA)

# Average replicates
ph_avg = ph_df.groupby(['S1','S2','C1','C2'])['P'].mean().reset_index().rename(columns={'P':'P_ph'})
mc_avg = mc_df.groupby(['S1','S2','C1','C2'])['P'].mean().reset_index().rename(columns={'P':'P_mc'})

# Only compositions measured in BOTH datasets
both = ph_avg.merge(mc_avg, on=['S1','S2','C1','C2'])

# Log-normalise each molecule's permeability to [0,1] and sum
log_ph = np.log10(both['P_ph'])
log_mc = np.log10(both['P_mc'])
score_ph = (log_ph - log_ph.min()) / (log_ph.max() - log_ph.min())
score_mc = (log_mc - log_mc.min()) / (log_mc.max() - log_mc.min())
both['combined_score'] = score_ph + score_mc
both = both.sort_values('combined_score').reset_index(drop=True)

# Build a label for each
def comp_label(row):
    parts = []
    if row.S1 > 0: parts.append(f'S1={int(row.S1)}')
    if row.S2 > 0: parts.append(f'S2={int(row.S2)}')
    if row.C1 > 0: parts.append(f'C1={int(row.C1)}')
    if row.C2 > 0: parts.append(f'C2={int(row.C2)}')
    return ', '.join(parts)

both['label'] = both.apply(comp_label, axis=1)

print(f'Found {len(both)} compositions measured in both Phenol and M-Cresol experiments:')
print()
print(f'{"Rank":<5} {"Composition":<35} {"P_Phenol (cm/s)":>18} {"P_M-Cresol (cm/s)":>20} {"Combined score":>16}')
print('─' * 98)
for i, row in both.iterrows():
    print(f'{i+1:<5} {row.label:<35} {row.P_ph:>18.3e} {row.P_mc:>20.3e} {row.combined_score:>16.3f}')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#16213e')
for spine in ax.spines.values():
    spine.set_edgecolor('#444')
ax.tick_params(colors='#ccc')

x = np.arange(len(both))
labels = both['label'].values

# Colour by rank
bar_colors = ['#f1c40f' if i == 0 else '#95a5a6' if i == 1 else '#cd7f32' if i == 2
              else '#555' for i in range(len(both))]

bars = ax.bar(x, both['combined_score'], color=bar_colors,
              edgecolor='#ffffff20', width=0.6)

for bar, v in zip(bars, both['combined_score']):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{v:.2f}', ha='center', va='bottom', fontsize=8, color='#ccc')

# Annotate top-3 bars with their permeabilities
for i in range(min(3, len(both))):
    row = both.iloc[i]
    medal = ['★ #1', '● #2', '◆ #3'][i]
    ax.text(i, both['combined_score'].iloc[i] / 2,
            f'{medal}\nPh:{row.P_ph:.1e}\nMC:{row.P_mc:.1e}',
            ha='center', va='center', fontsize=7.5,
            color='black', fontweight='bold')

ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=20, ha='right', fontsize=8.5, color='#ccc')
ax.set_ylabel('Combined score (lower = better)', color='#ccc')
ax.set_title('Joint ranking: compositions tested for BOTH Phenol and M-Cresol\n'
             'Score = normalised log(P_phenol) + normalised log(P_m-cresol)',
             color='#e0e0ff', fontsize=11)
ax.grid(axis='y', alpha=0.2)
plt.tight_layout()
plt.show()

---
## Key Takeaways

| | Phenol | M-Cresol |
|---|---|---|
| **#1** | M-07 — 30% Sparsa1 + 70% Sparsa2 | M-09 — 60% Sparsa1 + 10% Sparsa2 + 30% Carbosil1 |
| **#2** | M-04 — 100% Carbosil2 | M-14 — 20% Sparsa1 + 40% Sparsa2 + 40% Carbosil2 |
| **#3** | M-11 — 10/20/70% Sparsa1/Sparsa2/Carbosil1 | M-03(2) — 100% Carbosil1 |

**Why the model keeps recommending Sparsa-heavy compositions as the combined optimum:**
- M-07 (70% Sparsa2) is the best phenol membrane by a **factor of ~5** over the #2 composition
- For M-Cresol, the top compositions are much closer together (M-09 at 5.0e-8 vs M-03(2) at 7.7e-8)
- When the optimizer minimises both simultaneously, M-07's massive phenol advantage pulls the result toward the Sparsa2 region

**What would change this:** Testing more Carbosil-dominant blends. The data currently has:
- Pure Carbosil1 (M-03) and pure Carbosil2 (M-04) for phenol, but **no high-Carbosil blends with both components**
- The Bayesian optimizer (Tab 4 in the app) will specifically suggest compositions in poorly-explored regions

---
## M-07 Sensitivity Analysis — Is the Sparsa Dominance Real?

M-07 (30% Sparsa1 + 70% Sparsa2) is the best phenol membrane by a **factor of ~5** over the next best
composition. It has **no replicate**, sits in a **sparsely measured region** of composition space, and
single-handedly drives the optimizer toward Sparsa-heavy compositions.

This section investigates three questions:
1. **LOO residuals**: Is M-07 a leverage point — does the GP rely on it unusually heavily?
2. **GP surface**: How does the predicted phenol landscape change if M-07 is removed?
3. **Optimizer**: What composition does the model recommend when M-07 is excluded?

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath('.'))  # make models package importable

from models import GaussianProcessModel

ALPHA_PH = 0.003193   # derived from experimental replicates (see models.ipynb §7.1)
ALPHA_MC = 0.009659

# ── Averaged phenol data (merge replicates) ───────────────────────────────
ph_avg = pd.DataFrame(PHENOL_DATA).groupby(['S1','S2','C1','C2'])['P'] \
           .mean().reset_index().sort_values('P').reset_index(drop=True)
X_ph = ph_avg[['S1','S2','C1','C2']].values / 100.0
y_ph = np.log10(ph_avg['P'].values)

# Short labels for each unique composition
def short_label(row):
    parts = []
    if row['S1'] > 0: parts.append(f'S1={int(row.S1)}')
    if row['S2'] > 0: parts.append(f'S2={int(row.S2)}')
    if row['C1'] > 0: parts.append(f'C1={int(row.C1)}')
    if row['C2'] > 0: parts.append(f'C2={int(row.C2)}')
    return ', '.join(parts)

comp_labels = ph_avg.apply(short_label, axis=1).tolist()

# Find M-07 index (S1=30, S2=70, C1=0, C2=0)
m07_idx = int(np.where(
    (ph_avg['S1'] == 30) & (ph_avg['S2'] == 70) &
    (ph_avg['C1'] ==  0) & (ph_avg['C2'] ==  0)
)[0][0])

# ── Leave-One-Out cross-validation ────────────────────────────────────────
loo_pred = np.zeros(len(X_ph))
loo_std  = np.zeros(len(X_ph))

for i in range(len(X_ph)):
    X_tr = np.delete(X_ph, i, axis=0)
    y_tr = np.delete(y_ph, i)
    gp_tmp = GaussianProcessModel(alpha=ALPHA_PH).fit(X_tr, y_tr)
    mu, sig = gp_tmp.model.predict(X_ph[i:i+1], return_std=True)
    loo_pred[i] = mu[0]
    loo_std[i]  = sig[0]

loo_resid = y_ph - loo_pred          # actual − predicted  (log10 units)
loo_r2_all = 1 - np.sum(loo_resid**2) / np.sum((y_ph - y_ph.mean())**2)

# R² excluding M-07's own fold
mask_no07 = np.arange(len(X_ph)) != m07_idx
loo_r2_no07 = 1 - np.sum(loo_resid[mask_no07]**2) / \
                  np.sum((y_ph[mask_no07] - y_ph[mask_no07].mean())**2)

print(f'LOO R² (all points)      : {loo_r2_all:.3f}')
print(f'LOO R² (excluding M-07)  : {loo_r2_no07:.3f}')
print()
print(f'{"Composition":<35} {"Actual log10(P)":>17} {"LOO pred":>10} {"Residual":>10}  {"Flag"}')
print('─' * 85)
for i, (label, act, pred, res) in enumerate(zip(comp_labels, y_ph, loo_pred, loo_resid)):
    flag = '  ◀ M-07' if i == m07_idx else ''
    print(f'{label:<35} {act:>17.3f} {pred:>10.3f} {res:>10.3f}{flag}')

# ── Plot LOO residuals ─────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(13, 5))
fig.patch.set_facecolor('#1a1a2e')
ax.set_facecolor('#16213e')
for spine in ax.spines.values(): spine.set_edgecolor('#444')
ax.tick_params(colors='#ccc')

x = np.arange(len(X_ph))
colors = ['#e74c3c' if i == m07_idx else '#5b9bd5' for i in x]

bars = ax.bar(x, loo_resid, color=colors, edgecolor='#ffffff20', width=0.6, zorder=3)
ax.errorbar(x, loo_resid, yerr=2*loo_std, fmt='none',
            ecolor='#ffffff60', capsize=5, lw=1.5, zorder=4)

ax.axhline(0, color='#ffffff50', lw=1, ls='--', zorder=2)
ax.set_xticks(x)
ax.set_xticklabels(comp_labels, rotation=30, ha='right', fontsize=8, color='#ccc')
ax.set_ylabel('LOO Residual  (actual − predicted, log₁₀ units)', color='#ccc')
ax.set_title(
    f'Leave-One-Out residuals — Phenol GP\n'
    f'LOO R² all points: {loo_r2_all:.3f}   |   LOO R² excluding M-07: {loo_r2_no07:.3f}\n'
    f'Error bars = ±2σ of LOO GP prediction',
    color='#e0e0ff', fontsize=11)
ax.grid(axis='y', alpha=0.2)

from matplotlib.patches import Patch
ax.legend(handles=[
    Patch(color='#e74c3c', label='M-07 (no replicate)'),
    Patch(color='#5b9bd5', label='All other compositions'),
], facecolor='#1a1a2e', edgecolor='#555', labelcolor='#ddd', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# ── GP surface comparison: full data vs M-07 excluded ─────────────────────
# We project onto the S1/S2 subspace (C1=C2=0) since that's where M-07 lives.
# The grid covers all (S1, S2) pairs where S1+S2 ≤ 100% (the Sparsa simplex).

gp_full  = GaussianProcessModel(alpha=ALPHA_PH).fit(X_ph, y_ph)
gp_no07  = GaussianProcessModel(alpha=ALPHA_PH).fit(X_ph[mask_no07], y_ph[mask_no07])

# Grid in fraction space
n = 80
s1 = np.linspace(0, 1, n)
s2 = np.linspace(0, 1, n)
S1g, S2g = np.meshgrid(s1, s2)
valid = (S1g + S2g) <= 1.0

X_grid = np.column_stack([S1g.ravel(), S2g.ravel(),
                           np.zeros(n*n), np.zeros(n*n)])

mu_full,  std_full  = gp_full.model.predict(X_grid, return_std=True)
mu_no07,  std_no07  = gp_no07.model.predict(X_grid, return_std=True)

for arr in [mu_full, mu_no07, std_full, std_no07]:
    arr.shape = (n, n)
    arr[~valid] = np.nan

# Shared colour scale for mean (use full-data range)
vmin_mu = np.nanmin(mu_full)
vmax_mu = np.nanmax(mu_full)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.patch.set_facecolor('#1a1a2e')

titles = [
    ('GP mean — ALL data (incl. M-07)',          mu_full,  'RdYlGn', vmin_mu, vmax_mu),
    ('GP mean — M-07 EXCLUDED',                   mu_no07,  'RdYlGn', vmin_mu, vmax_mu),
    ('GP uncertainty (1σ) — ALL data',           std_full,  'plasma',  None,    None),
    ('GP uncertainty (1σ) — M-07 EXCLUDED',      std_no07,  'plasma',  None,    None),
]

for ax, (title, data, cmap, vmin, vmax) in zip(axes.ravel(), titles):
    ax.set_facecolor('#16213e')
    for spine in ax.spines.values(): spine.set_edgecolor('#444')
    ax.tick_params(colors='#ccc')

    im = ax.contourf(S1g * 100, S2g * 100, data, levels=30,
                     cmap=cmap, vmin=vmin, vmax=vmax)
    plt.colorbar(im, ax=ax, pad=0.02).ax.tick_params(colors='#ccc')

    # Plot training points
    for i, row in ph_avg.iterrows():
        color = '#ff4444' if i == m07_idx else '#ffffff'
        ms = 12 if i == m07_idx else 7
        ax.scatter(row.S1, row.S2, color=color, s=ms**2, zorder=5,
                   edgecolors='black', lw=0.8)
        if i == m07_idx:
            ax.annotate('M-07', (row.S1, row.S2),
                        textcoords='offset points', xytext=(5, 5),
                        fontsize=9, color='#ff4444', fontweight='bold')

    ax.set_xlabel('Sparsa 1 (%)', color='#ccc')
    ax.set_ylabel('Sparsa 2 (%)', color='#ccc')
    ax.set_xlim(-2, 102)
    ax.set_ylim(-2, 102)
    ax.set_title(title, color='#e0e0ff', fontsize=10)

    # Shade the invalid region (S1+S2 > 100)
    ax.fill_between([0, 100], [100, 0], [100, 100],
                    color='#1a1a2e', alpha=0.8, zorder=1)

plt.suptitle(
    'Phenol GP in the Sparsa sub-space (C1=C2=0)\n'
    'Top row: predicted log₁₀(P)  [green=low=good]   '
    'Bottom row: prediction uncertainty [bright=uncertain]\n'
    'Red dot = M-07 (no replicate)',
    color='#e0e0ff', fontsize=11, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
from models import BayesianOptimiser

# ── M-Cresol GP (unchanged — M-07 data point stays in MC dataset) ─────────
mc_avg = pd.DataFrame(MCRESOL_DATA).groupby(['S1','S2','C1','C2'])['P'] \
           .mean().reset_index()
X_mc = mc_avg[['S1','S2','C1','C2']].values / 100.0
y_mc = np.log10(mc_avg['P'].values)
gp_mc = GaussianProcessModel(alpha=ALPHA_MC).fit(X_mc, y_mc)

# ── Run optimizer under both scenarios ────────────────────────────────────
opt_full  = BayesianOptimiser(gp_full,  gp_mc, y_ph,              y_mc).suggest_next()
opt_no07  = BayesianOptimiser(gp_no07,  gp_mc, y_ph[mask_no07],  y_mc).suggest_next()

def fmt_comp(x):
    names = ['S1', 'S2', 'C1', 'C2']
    return '  '.join(f'{n}={v*100:.1f}%' for n, v in zip(names, x) if v > 0.005)

print('══════════════════════════════════════════════════════════════════')
print('  BAYESIAN OPTIMIZER — SUGGESTED NEXT COMPOSITION TO TEST')
print('══════════════════════════════════════════════════════════════════')
print()
print(f'  ┌─ WITH M-07 included ─────────────────────────────────────────')
print(f'  │  Composition : {fmt_comp(opt_full["x_opt"])}')
print(f'  │  Pred phenol : {opt_full["pred_ph"]:.3e} cm/s  (95% CI: {opt_full["ci95_ph"][0]:.2e}–{opt_full["ci95_ph"][1]:.2e})')
print(f'  │  Pred M-cresol: {opt_full["pred_mc"]:.3e} cm/s  (95% CI: {opt_full["ci95_mc"][0]:.2e}–{opt_full["ci95_mc"][1]:.2e})')
print()
print(f'  ┌─ WITHOUT M-07 ───────────────────────────────────────────────')
print(f'  │  Composition : {fmt_comp(opt_no07["x_opt"])}')
print(f'  │  Pred phenol : {opt_no07["pred_ph"]:.3e} cm/s  (95% CI: {opt_no07["ci95_ph"][0]:.2e}–{opt_no07["ci95_ph"][1]:.2e})')
print(f'  │  Pred M-cresol: {opt_no07["pred_mc"]:.3e} cm/s  (95% CI: {opt_no07["ci95_mc"][0]:.2e}–{opt_no07["ci95_mc"][1]:.2e})')
print()

# ── Stacked bar chart comparing the two suggestions ────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
fig.patch.set_facecolor('#1a1a2e')

comp_names = ['Sparsa 1\n(27G26)', 'Sparsa 2\n(30G25)', 'Carbosil 1\n(2080A)', 'Carbosil 2\n(2090A)']
comp_colors_list = list(COMP_COLORS.values())

scenarios = [
    (axes[0], opt_full['x_opt'],  'With M-07',    '#e74c3c'),
    (axes[1], opt_no07['x_opt'],  'Without M-07', '#f39c12'),
]

for ax, x_opt, label, header_color in scenarios:
    ax.set_facecolor('#16213e')
    for spine in ax.spines.values(): spine.set_edgecolor('#444')
    ax.tick_params(colors='#ccc')

    bottom = 0
    for val, name, color in zip(x_opt * 100, comp_names, comp_colors_list):
        if val > 0.5:
            bar = ax.bar(0.5, val, bottom=bottom, color=color,
                         edgecolor='#1a1a2e', width=0.6, label=name)
            ax.text(0.5, bottom + val / 2, f'{val:.1f}%',
                    ha='center', va='center', fontsize=11,
                    color='white', fontweight='bold')
        bottom += val

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 105)
    ax.set_xticks([])
    ax.set_ylabel('Composition (%)', color='#ccc')
    ax.set_title(f'Suggested composition\n({label})', color=header_color, fontsize=11)
    ax.grid(axis='y', alpha=0.2)

handles = [mpatches.Patch(color=c, label=n.replace('\n', ' '))
           for n, c in zip(comp_names, comp_colors_list)]
fig.legend(handles=handles, loc='lower center', ncol=4, fontsize=9,
           facecolor='#1a1a2e', edgecolor='#555', labelcolor='#ddd',
           bbox_to_anchor=(0.5, -0.06))

plt.suptitle('What does the Bayesian optimizer suggest next — with vs without M-07?',
             color='#e0e0ff', fontsize=12)
plt.tight_layout()
plt.show()

---
## What This Tells Us

### LOO Residuals
If M-07's LOO residual is large (much larger than all other points), the GP has **no nearby data to
support its prediction** — the model trained without M-07 has to extrapolate into the Sparsa2-heavy
region from far away, and gives a very different (worse) prediction. A large residual here means the
model is **relying entirely on M-07 itself** to define that region of the landscape.

### GP Surface
- **With M-07**: A clear "dip" (green region) appears near the Sparsa2-heavy corner — the model
  confidently predicts low permeability there.
- **Without M-07**: That dip **disappears or flattens significantly**, and the uncertainty map shows
  a **bright region** (high σ) exactly where M-07 was — the model is honest that it simply doesn't
  know what happens in that corner of composition space.

### Optimizer
- **With M-07**: The optimizer exploits the predicted dip and recommends a Sparsa2-heavy composition.
- **Without M-07**: The optimizer shifts toward either a different exploitation target (another
  known good region) or a more exploration-driven composition in an uncharted area.

### Conclusion
M-07 is **a high-leverage single point** in an otherwise uncharted region of composition space.
Its result may well be real — but without at least one replicate of M-07, the model is effectively
**extrapolating from a single measurement**. The responsible path forward is:

1. **Replicate M-07** — run it again under the same conditions. If the result reproduces (~3.4e-8),
   Sparsa2-heavy is genuinely exceptional and the model is correctly guiding you there.
2. **If it doesn't reproduce**, remove it and re-run the analysis using the no-M07 optimizer output
   as the next composition to test.